In [ ]:
import pandas as pd 

df = pd.read_json('data\data_ebay.json', lines=True)

In [ ]:
def print_nans_perc(df, Print = False, perc = 0.95):
    n = len(df)
    nans_perc = df.isna().sum() / len(df)
    cols = nans_perc[nans_perc > 0.95].index
    print('len - ', n)
    nans_perc.sort_values(ascending=True, inplace=True)
    if Print:
        for index, val in zip(nans_perc.index, nans_perc):
            print(index + ': ', val)
    return cols
print_nans_perc(df, Print = True)

In [ ]:
def drop_nans_in_cols(df : pd.DataFrame, cols : list):
    print(len(df))
    df.dropna(subset=cols, inplace=True)
    df.dropna(axis = 1, how = 'all', inplace=True)
    print(len(df))
    print_nans_perc(df)

In [ ]:
df['type'] = df['Bike Type'].combine_first(df['Type'])
df.drop(['Bike Type', 'Type'], axis = 1, inplace=True)
print('perc of left nans: ', df['type'].isna().sum() / len(df))

In [ ]:
df['Price'].replace('[A-Za-z, \$, £]','',inplace=True, regex=True)
df = df[df['Price'] != '']
df['Price'] = df['Price'].astype(float)
df[df['Price'] < 2500]['Price'].plot(kind='hist')

In [ ]:
bike_groups = [
    "Dura Ace", "Dura-Ace", "Ultegra", "105", "Tiagra", "Sora", "Claris", "GRX",
    
    "XTR", "Deore XT", "SLX", "Deore", "Alivio", "Acera", "Altus", "Tourney", "Zee", "Saint",
    
    "Super Record", "Record", "Chorus", "Potenza", "Centaur", "Veloce", "Athena",
    
    "Sram Red", "Force", "Rival", "Apex",

    "XX1", "X01","GX", "NX", "SX", "XX", "X0", "X9", "X7", "X5", "Descendant", "Code"
]

road_groups = [ "dura ace", "dura-ace", "ultegra", "105", "tiagra", "sora", "claris", "grx",
               "super record", "record", "chorus", "potenza", "centaur", "veloce", "athena",
                "sram red", "Force", "arval", "apex"]

mtb_groups = [
    "xtr", "deore xt", "slx", "deore", "alivio", "acera", "altus", "tourney", "zee", "saint",
    "xx1", "x01", "gx", "nx", "sx", "xx", "x0", "x9", "x7", "x5", "descendant", "code"
]

bike_groups_dict_lower = {
    "dura ace": 1.0, "dura-ace": 1.0, "ultegra": 0.9, "105": 0.8, "tiagra": 0.7, "sora": 0.6, "claris": 0.5, "grx": 0.8,
    
    "xtr": 1.0, "deore xt": 0.9, "slx": 0.8, "deore": 0.7, "alivio": 0.6, "acera": 0.5, "altus": 0.4, "tourney": 0.3, "zee": 0.8, "saint": 0.9,
    
    "super record": 1.0, "record": 0.9, "chorus": 0.8, "potenza": 0.7, "centaur": 0.6, "veloce": 0.5, "athena": 0.4,
    
    "sram red": 1.0, "force": 0.9, "rival": 0.8, "apex": 0.7,
    
    "xx1": 1.0, "x01": 0.9, "gx": 0.8, "nx": 0.7, "sx": 0.6, "xx": 0.9, "x0": 0.8, "x9": 0.7, "x7": 0.6, "x5": 0.5, "descendant": 0.7, "code": 0.7
}


df.drop(['Color', 'Colour', 'Product URL', 'Photo URL'], axis=1, inplace=True)

In [ ]:
import re
import numpy as np

def find_from_list(row, columns, list):
    for col in columns:
        value = row[col]
        found = ''
        if pd.notna(value) and isinstance(value, str): 
            found = re.findall(r'(' + '|'.join(list) + ')', value, re.IGNORECASE)
        if found:
            return str(found[0]).lower()
    return np.nan

cols = ['Title', 'Seller notes', 'Model', 'Brake Type']
#cols = df.columns
df['groupset'] = np.nan
df['groupset'] = df.apply(find_from_list, axis=1, args = (cols, bike_groups))
print('perc of left nans: ', df['groupset'].isna().sum() / len(df))
df['groupset_num'] = df['groupset'].replace(bike_groups_dict_lower, regex=True)
df['groupset_num'] = df['groupset_num'].astype(float)

In [ ]:
df['speeds'] = df['Number of Speeds'].combine_first(df['Number of Gears'])
df['speeds_num'] = df['speeds'].str.extract(r'\b(\d{1,2})\b').astype(float)

def re_count_gears(x):
    if x < 5 or x > 33:
        return np.nan
    elif x <= 12:
        return x
    else:
        if x % 2 == 0:
            if x/2 > 12:
                return np.nan
            else:
                return x/2
        elif x % 3 == 0:
            return x/3

speeds_dict = {5: 0.2, 6: 0.3, 7: 0.4, 8: 0.5, 9: 0.6, 10: 0.7, 11: 0.8, 12: 0.9}

df['speeds_num'] = df['speeds_num'].apply(re_count_gears)
df['speeds_num'].replace(speeds_dict, regex=True, inplace=True)
print('perc of left nans: ', df['groupset_num'].isna().sum() / len(df))
df['groupset_num'] = df['groupset_num'].combine_first(df['speeds_num']) 
print('perc of left nans: ', df['groupset_num'].isna().sum() / len(df))

In [ ]:
materials = ['carbon', 'alu', 'aluminium', 'steel', 'alloy', 'titanium', 'chromoly']
materials_dict = {'carbon': 0.9, 'alu': 0.5, 'aluminium': 0.5, 'steel': 0.3, 'alloy': 0.5, 'titanium': 0.6, 'chromoly': 0.15}

cols = df.columns
df['material'] = np.nan
df['material'] = df.apply(find_from_list, axis=1, args = (cols, materials,))
print('perc of left nans: ', df['material'].isna().sum() / len(df))
df['material_num'] = df['material'].replace(materials_dict, regex=True)
df['material_num'] = df['material_num'].astype(float)
df.groupby('material').size().sort_values()

In [ ]:
e_shift = ['di2', 'axs', 'etap']
e_shift_dict = {'di2': 1, 'axs': 1, 'etap': 1, np.nan: 0}

df['e_shift_type'] = np.nan 
cols = df.columns
df['e_shift_type'] = df.apply(find_from_list, axis=1, args=(cols, e_shift,))
print('perc of left nans: ', df['e_shift_type'].isna().sum() / len(df))
df['e_shift_num'] = df['e_shift_type'].replace(e_shift_dict, regex=True)
df['e_shift_num'] = df['e_shift_num'].astype(float)
df.groupby('e_shift_num').size().sort_values()

In [ ]:
df = df[df['Condition'] != 'For parts or not working']
condition_dict = {'Used': 0, 'New': 1, 'New other (see details)': 1, 'Seller refurbished': 0, 'Manufacturer refurbished': 0, '--': 0}

df['isNew'] = df['Condition'].replace(condition_dict, regex=True)
df['isNew'] = df['isNew'].astype(float)
print('perc of left nans: ', df['isNew'].isna().sum() / len(df))
df.groupby('isNew').size().sort_values()

In [ ]:
df.drop(['MPN'], axis=1, inplace=True)

In [ ]:
def find_year(row, cols):
    for col in cols:
        value = row[col]
        if pd.notna(value) and isinstance(value, str):
            four_num = re.findall(r'\b[0-9]{4}\b', value)
            four_num = [float(num) for num in four_num]
            for num in four_num:
                if 1950 <= num <= 2024:
                    return num
    return np.nan

df['Production_year'] = np.nan
cols = ['Model Year','Title'] 
#cols = df.columns
df['Production_year'] = df.apply(find_year, axis=1, args=(cols,))
print('perc of left nans: ', df['Production_year'].isna().sum() / len(df))
'''#print(df[df['groupset'] == 'deore xt'][['Title', 'Production_year']])
df[df['Production_year'] < 2000].groupby('groupset').size()
df.groupby('groupset').size().sort_values()
df.groupby('groupset')['Production_year'].median()[df.groupby('groupset')['Production_year'].median() < 2010]
'''

In [ ]:
brakes = ['rim', 'disc', 'v-brake', 'hydraulic', 'caliper', 'cantilever', 'v-brakes']
brakes_dict = {'rim': 0.5, 'disc': 1, 'v-brake': 0.5, 'hydraulic': 1, 'caliper': 0.5, 'cantilever': 0.5, 'v-brakes': 0.5}

df['brake_type'] = np.nan 
cols = df.columns
df['brake_type'] = df.apply(find_from_list, axis=1, args=(cols, brakes,))
print('perc of left nans: ', df['brake_type'].isna().sum() / len(df))
df['brake_num'] = df['brake_type'].replace(brakes_dict, regex=True)
df['brake_num'] = df['brake_num'].astype(float)
df.groupby('brake_num').size().sort_values()

In [ ]:
df[df['groupset'] == 'ultegra']['Production_year'].plot(kind='hist')
df[df['groupset'] == '105']['Production_year'].plot(kind='hist')
df[df['groupset'] == 'dura ace']['Production_year'].plot(kind='hist')
df[df['groupset'] == 'sram red']['Production_year'].plot(kind='hist')
df.groupby('groupset')['Production_year'].median()[df.groupby('groupset').size() > 100]

In [ ]:
print('perc od left nans: ', df['Production_year'].isna().sum() / len(df))
df.loc[df['groupset'] == '105', 'Production_year'] = df[df['groupset'] == '105']['Production_year'].fillna(2017)
df.loc[df['groupset'] == 'ultegra', 'Production_year'] = df[df['groupset'] == 'ultegra']['Production_year'].fillna(2018)
df.loc[df['groupset'] == 'dura ace', 'Production_year'] = df[df['groupset'] == 'dura ace']['Production_year'].fillna(2017)
df.loc[df['groupset'] == 'sram red', 'Production_year'] = df[df['groupset'] == 'sram red']['Production_year'].fillna(2018)
print('perc od left nans: ', df['Production_year'].isna().sum() / len(df))

In [ ]:
print('perc od left nans: ', df['type'].isna().sum() / len(df))
df.loc[df['groupset'].isin(road_groups) & df['type'].isna(), 'type'] = 'Road Bike'
df.loc[df['groupset'].isin(mtb_groups) & df['type'].isna(), 'type'] = 'mtb'
print('perc od left nans: ', df['type'].isna().sum() / len(df))


In [ ]:
print('perc od left nans: ', df['Production_year'].isna().sum() / len(df))
indexes = df['Condition'].isin(['New', 'New other (see details)'])
df.loc[indexes, 'Production_year'] = df.loc[indexes, 'Production_year'].fillna(2019) # 2019 - median of new bikes
print('perc od left nans: ', df['Production_year'].isna().sum() / len(df))

In [ ]:
cols = print_nans_perc(df)
to_drop = ['Frame Size', 'Department', 'UPC', 'Gender', 'Shifter Style', 
              'Features', 'Country/Region of Manufacture', 'Tyre Type', 'Style', 'Manufacturer Color', 'Tire Type',
              'Modified Item', 'Tires', 'Custom-Built', 'Custom Bundle', 'Handlebar Type', 'Shifter', 'Gear Change Mechanism', 'e_shift_type', 
              'Model Year', 'Configuration', 'Frame Material', 'Brake Type', 'Condition', 'Number of Gears', 'speeds', 'Number of Speeds', 
              'Material', 'speeds_num', 'Vintage', 'Model', 'Brand']
to_drop += list(cols)

df_drp = df.drop(to_drop, axis = 1, inplace=False)
print_nans_perc(df_drp, Print=True)

In [ ]:
roadies = df_drp[df_drp['type'].isin(['Road Bike', 'Road Bike - Racing', 'Road bike', 'Road Bike - Touring', 
                              'Road', 'Cyclocross Bike', 'cyclocross/gravel', 'ROAD DISC BIKE', 'Road Bike - Racing or Touring', 
                              'Aero Road Bike', 'Racing/Touring Roadbike', 'Road/Touring', 'Time Trial/Triathlon Bike', 
                              'Road Bike Racing', 'Time Trial/Triathlon', 'Road/Racing', 'Racing/road bike', 
                              'Raod Bike', 'Road Racing Bike', 'road bike', 'Road Bike - Touring ??', 
                              'Road Bike / triathlon', 'Racing/Road Bike', 'Comfort Road Bike',  
                              'road Bike', 'Road Bike / Racing Bike', 'Hybrid', 
                              'Gravel, CX, Road, Commuter', 'Road Bike / Cyclocross Bike / Triathlon', 'Race Road Bike',
])]

cols_to_drop_road = ['Wheel Size', 'Suspension', 'Suspension Type', 'Seller notes']
roadies.drop(columns=cols_to_drop_road, inplace=True)

roadies['type'] = 'Road Bike'

In [ ]:
old_idx = roadies.loc[roadies['Production_year'] < 2015].index
roadies.loc[old_idx, 'brake_num'] = 0.5

In [ ]:
roadies_num = roadies.drop(columns=['groupset', 'brake_type', 'material', 'type', 'Title'])
print_nans_perc(roadies_num, Print=True)
print_nans_perc(roadies_num.dropna(axis=0, how='any'), Print=True)

In [ ]:
roadies_num

In [ ]:
mtb = df_drp[df_drp['type'].isin(['Mountain Bike', 'Full Suspension', 'Downhill Bike', 'MOUNTAIN BIKE', 'Mountain Bike/ Gravel', 
                          'Mountain Bike MTB 29ER', 'Mountain Suspension Bike', 'Hardtail Bike','Mountain bike', 'mountain  bike style', 
                          'Hardtail Mountain Bike', 'MOUNTAIN', 'mountain bike', 'Mountain Bike19"', 'Enduro', '29er Mountain Bike', 'Mountain / JUMP Bike',  
])]

mtb['type'] = 'MTB'

print_nans_perc(mtb, Print=True)

In [ ]:
wheel_size = ['26', '27.5', '27', '27.5', '29']
wheel_size_dict = {'27.5': 0.66, '29': 1, '27': 0.66, '27,5': 0.66, '26': 0.33}

cols = ['Wheel Size']
mtb['Wheel Size'] = mtb.apply(find_from_list, axis=1 ,args=(cols, wheel_size, ))
mtb['wheel_num'] = mtb['Wheel Size'].replace(wheel_size_dict, regex=True)
print_nans_perc(mtb, Print=True)

In [ ]:
mtb['suspension'] = mtb['Suspension'].combine_first(mtb['Suspension Type'])
mtb.drop(columns=['Suspension', 'Suspension Type'], inplace=True)

In [ ]:
susps = ['front', 'full', 'no suspension', 'rear']
susps_dict = {'front': 0.5, 'full': 1, 'no': 0, 'rear': 1}

cols = mtb.columns
mtb['suspension'] = mtb.apply(find_from_list, axis=1 ,args=(cols, susps, ))
mtb['suspension_num'] = mtb['suspension'].replace(susps_dict, regex=True)
print_nans_perc(mtb, Print=True)

In [ ]:
old_idx = mtb.loc[mtb['Production_year'] > 2010].index
mtb.loc[old_idx, 'brake_num'] = 1

In [ ]:
mtb_num = mtb.drop(columns=['groupset', 'brake_type', 'material', 'type', 'Wheel Size', 'Seller notes', 'Title', 'suspension'])
print_nans_perc(mtb_num, Print=True)
print_nans_perc(mtb_num.dropna(axis=0, how='any'))


In [ ]:
roadies_num.dropna(axis=0, how='any', inplace=True)
num_cols = ['ID', 'Price','Production_year', 'isNew', 'material_num', 'groupset_num', 'e_shift_num', 'brake_num']
roadies_num = roadies_num[num_cols]
roadies_num.to_json('data\ebay_road.json', lines=True, orient='records')
roadies_num

In [ ]:
mtb_num.dropna(axis=0, how='any', inplace=True)
num_cols = ['ID', 'Price','Production_year', 'isNew', 'suspension_num','material_num', 'groupset_num', 'e_shift_num', 'wheel_num', 'brake_num']
mtb_num = mtb_num[num_cols]
mtb_num.to_json('data\ebay_mtb.json', lines=True, orient='records')
mtb_num